In [1]:
import csv
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import warnings

import tensorflow as tf
import tensorflow_probability as tfp

In [2]:
import urllib
import os
os.getcwd()

'/Users/yafeng/Documents/tfp'

In [3]:
# # We'll use the following directory to store files we download as well as our
# # preprocessed dataset.
# CACHE_DIR = os.path.join(os.sep, '/projects/bbr/code_yc/tfp/test_data/tmp', 'radon')


# def cache_or_download_file(cache_dir, url_base, filename):
#     """Read a cached file or download it."""
#     filepath = os.path.join(cache_dir, filename)
#     if tf.gfile.Exists(filepath):
#         return filepath
#     if not tf.gfile.Exists(cache_dir):
#         tf.gfile.MakeDirs(cache_dir)
#     url = os.path.join(url_base, filename)
#     print("Downloading {url} to {filepath}.".format(url=url, filepath=filepath))
#     urllib.request.urlretrieve(url, filepath)
#     return filepath


# def download_radon_dataset(cache_dir=CACHE_DIR):
#     """Download the radon dataset and read as Pandas dataframe."""
#     url_base = 'http://www.stat.columbia.edu/~gelman/arm/examples/radon/'
#     # Alternative source:
#     # url_base = ('https://raw.githubusercontent.com/pymc-devs/uq_chapter/'
#     #             'master/reference/data/')
#     srrs2 = pd.read_csv('test_data/srrs2.csv')
#     srrs2.rename(columns=str.strip, inplace=True)
#     cty = pd.read_csv('test_data/cty.csv')
#     cty.rename(columns=str.strip, inplace=True)
#     return srrs2, cty


# def preprocess_radon_dataset(srrs2, cty, state='MN'):
#     """Preprocess radon dataset as done in "Bayesian Data Analysis" book."""
#     srrs2 = srrs2[srrs2.state==state].copy()
#     cty = cty[cty.st==state].copy()

#     # We will now join datasets on Federal Information Processing Standards
#     # (FIPS) id, ie, codes that link geographic units, counties and county
#     # equivalents. http://jeffgill.org/Teaching/rpqm_9.pdf
#     srrs2['fips'] = 1000 * srrs2.stfips + srrs2.cntyfips
#     cty['fips'] = 1000 * cty.stfips + cty.ctfips

#     df = srrs2.merge(cty[['fips', 'Uppm']], on='fips')
#     df = df.drop_duplicates(subset='idnum')
#     df = df.rename(index=str, columns={'Uppm': 'uranium_ppm'})

#     # For any missing or invalid activity readings, we'll use a value of `0.1`.
#     df['radon'] = df.activity.apply(lambda x: x if x > 0. else 0.1)

#     # Remap categories to start from 0 and end at max(category).
#     county_name = sorted(df.county.unique())
#     df['county'] = df.county.astype(
#         pd.api.types.CategoricalDtype(categories=county_name)).cat.codes
#     county_name = map(str.strip, county_name)

#     df['radon'] = df['radon'].apply(np.log)
#     df['log_uranium_ppm'] = df['uranium_ppm'].apply(np.log) 
#     df = df[['radon', 'floor', 'county', 'log_uranium_ppm']]

#     return df, county_name

In [4]:
# CACHE_DIR

In [5]:
# with tf.gfilea.Open(os.path.join(CACHE_DIR, 'radon.csv'), 'w') as f:
#     radon.to_csv(f, index=False)

In [6]:
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
#     tf.reset_default_graph()
#     try:
#         sess.close()
#     except:
#         pass
#     sess = tf.InteractiveSession()

In [7]:
inv_scale_transform = lambda y: np.log(y)  # Not using TF here.
fwd_scale_transform = tf.exp

In [8]:
# def _make_weights_hyper_prior(dtype):
#     """Returns a `len(log_uranium_ppm)` batch of univariate Normal."""
#     raw_prior_scale = tf.get_variable(
#         name='raw_hyper_prior_scale',
#         initializer=np.array(inv_scale_transform(1.), dtype=dtype))
#     return tfp.distributions.InverseGamma(
#         rate=fwd_scale_transform(raw_prior_scale),
#         concentration=fwd_scale_transform(raw_prior_scale))


# make_weights_hyper_prior = tf.make_template(
#     name_='make_weights_hyper_prior', func_=_make_weights_hyper_prior)

In [9]:
# def _make_weights_prior(num_counties, dtype):
#     """Returns a `len(log_uranium_ppm)` batch of univariate Normal."""
#     raw_prior_scale = tf.get_variable(
#         name='raw_prior_scale',
#         initializer=np.array(inv_scale_transform(1.), dtype=dtype))
#     return tfp.distributions.Independent(
#         tfp.distributions.Normal(
#             loc=tf.zeros(num_counties, dtype=dtype),
#             scale=fwd_scale_transform(raw_prior_scale)),
#         reinterpreted_batch_ndims=1)


# make_weights_prior = tf.make_template(
#     name_='make_weights_prior', func_=_make_weights_prior)

In [10]:
# def _make_radon_likelihood(
#     random_effect_weights, floor, county,log_county_uranium_ppm, init_radon_stddev
# ):
#     raw_likelihood_scale = tf.get_variable(
#         name='raw_likelihood_scale',
#         initializer=np.array(
#             inv_scale_transform(init_radon_stddev), dtype=dtype))
#     fixed_effect_weights = tf.get_variable(
#         name='fixed_effect_weights', initializer=np.array([0., 1.], dtype=dtype))
#     fixed_effects = fixed_effect_weights[0] + fixed_effect_weights[1] * floor
#     random_effects = tf.gather(
#         random_effect_weights * log_county_uranium_ppm,
#         indices=tf.to_int32(county),
#         axis=-1)
#     linear_predictor = fixed_effects + random_effects
#     return tfp.distributions.Normal(
#         #rate=linear_predictor, concentration=fwd_scale_transform(raw_likelihood_scale))
#         loc=linear_predictor, scale=fwd_scale_transform(raw_likelihood_scale))


# make_radon_likelihood = tf.make_template(
#     name_='make_radon_likelihood', func_=_make_radon_likelihood)

In [11]:
# def joint_log_prob(
#     random_effect_weights, radon, floor, county,log_county_uranium_ppm, dtype
# ):
#     num_counties = len(log_county_uranium_ppm)
#     rv_weights = make_weights_prior(num_counties, dtype)
#     rv_radon = make_radon_likelihood(
#         random_effect_weights,
#         floor,
#         county,
#         log_county_uranium_ppm,
#         init_radon_stddev=radon.std())
#     return (
#         rv_weights.log_prob(random_effect_weights)+ \
#         tf.reduce_sum(rv_radon.log_prob(radon), axis=-1)
#     )

In [12]:
# # Specify unnormalized posterior.

# dtype = np.float32

# log_county_uranium_ppm = radon[
#     ['county', 'log_uranium_ppm']].drop_duplicates().values[:, 1]
# log_county_uranium_ppm = log_county_uranium_ppm.astype(dtype)

# def unnormalized_posterior_log_prob(random_effect_scale,random_effect_weights):
#     return joint_log_prob2(
#         my_sigma = random_effect_scale, 
#         my_gamma=random_effect_weights,
#         radon=dtype(radon.radon.values),
#         floor=dtype(radon.floor.values),
#         county=np.int32(radon.county.values),
#         log_county_uranium_ppm=log_county_uranium_ppm,
#         dtype=dtype
#     )

In [13]:
# # Set-up E-step.

# step_size_hmc = tf.get_variable(
#     'step_size_hmc',
#     initializer=np.array(0.2, dtype=dtype),
#     trainable=False)

# hmc = tfp.mcmc.HamiltonianMonteCarlo(
#     target_log_prob_fn=unnormalized_posterior_log_prob,
#     num_leapfrog_steps=2,
#     step_size=step_size_hmc,
#     step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(
#         num_adaptation_steps=None),
#     state_gradients_are_stopped=True)

# init_random_weights = tf.placeholder(dtype, shape=[len(log_county_uranium_ppm)])

# posterior_random_weights, kernel_results = tfp.mcmc.sample_chain(
#     num_results=3,
#     num_burnin_steps=0,
#     num_steps_between_results=0,
#     current_state=init_random_weights,
#     kernel=hmc)

In [14]:
# # Set-up M-step.

# loss = -tf.reduce_mean(kernel_results.accepted_results.target_log_prob)

# global_step = tf.train.get_or_create_global_step()

# learning_rate = tf.train.exponential_decay(
#     learning_rate=0.1,
#     global_step=global_step,
#     decay_steps=2,
#     decay_rate=0.99)

# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# train_op = optimizer.minimize(loss, global_step=global_step)

In [15]:
# init_op = tf.global_variables_initializer()

In [16]:
# # Grab variable handles for diagnostic purposes.

# with tf.variable_scope('make_weights_prior', reuse=True):
#     prior_scale = fwd_scale_transform(
#         tf.get_variable(
#         name='raw_prior_scale', dtype=dtype
#         ))

In [17]:
# with tf.variable_scope('make_radon_likelihood', reuse=True):
#     likelihood_scale = fwd_scale_transform(tf.get_variable(
#         name='raw_likelihood_scale', dtype=dtype))
#     fixed_effect_weights = tf.get_variable(
#         name='fixed_effect_weights', dtype=dtype)

In [18]:
# init_op.run()
# w_ = np.zeros([len(log_county_uranium_ppm)], dtype=dtype)

In [19]:
# maxiter = int(1500)
# num_accepted = 0
# num_drawn = 0
# for i in range(maxiter):
#     [
#         _,
#         global_step_,
#         loss_,
#         posterior_random_weights_,
#         kernel_results_,
#         step_size_hmc_,
#         prior_scale_,
#         likelihood_scale_,
#         fixed_effect_weights_,
#     ] = sess.run([
#         train_op,
#         global_step,
#         loss,
#         posterior_random_weights,
#         kernel_results,
#         step_size_hmc,
#         prior_scale,
#         likelihood_scale,
#         fixed_effect_weights,
#     ], feed_dict={init_random_weights: w_})
#     w_ = posterior_random_weights_[-1, :]
#     num_accepted += kernel_results_.is_accepted.sum()
#     num_drawn += kernel_results_.is_accepted.size
#     acceptance_rate = num_accepted / num_drawn
#     if i % 100 == 0 or i == maxiter - 1:
#         print(
#             'global_step:{:>4}  loss:{: 9.3f}  acceptance:{:.4f}  '
#             'step_size:{:.4f}  prior_scale:{:.4f}  likelihood_scale:{:.4f}  '
#             'fixed_effect_weights:{}'.format(
#                 global_step_, loss_.mean(), acceptance_rate, step_size_hmc_,
#                 prior_scale_, likelihood_scale_, fixed_effect_weights_))              

In [20]:
# posterior_random_weights_final, kernel_results_final = tfp.mcmc.sample_chain(
#     num_results=int(15e3),
#     num_burnin_steps=int(1e3),
#     current_state=init_random_weights,
#     kernel=tfp.mcmc.HamiltonianMonteCarlo(
#       target_log_prob_fn=unnormalized_posterior_log_prob,
#       num_leapfrog_steps=2,
#       step_size=step_size_hmc))

# [
#     posterior_random_weights_final_,
#     kernel_results_final_,
# ] = sess.run([
#     posterior_random_weights_final,
#     kernel_results_final,
# ], feed_dict={init_random_weights: w_})

In [21]:
# radon.radon.hist(bins=100)

In [22]:
def evaluate(tensors):
    """Evaluates Tensor or EagerTensor to Numpy `ndarray`s.
    Args:
    tensors: Object of `Tensor` or EagerTensor`s; can be `list`, `tuple`,
      `namedtuple` or combinations thereof.
 
    Returns:
      ndarrays: Object with same structure as `tensors` except with `Tensor` or
        `EagerTensor`s replaced by Numpy `ndarray`s.
    """
    if tf.executing_eagerly():
        return tf.contrib.framework.nest.pack_sequence_as(
            tensors,
            [t.numpy() if tf.contrib.framework.is_tensor(t) else t
             for t in tf.contrib.framework.nest.flatten(tensors)])
    return sess.run(tensors)


class _TFColor(object):
    """Enum of colors used in TF docs."""
    red = '#F15854'
    blue = '#5DA5DA'
    orange = '#FAA43A'
    green = '#60BD68'
    pink = '#F17CB0'
    brown = '#B2912F'
    purple = '#B276B2'
    yellow = '#DECF3F'
    gray = '#4D4D4D'
    def __getitem__(self, i):
        return [
            self.red,
            self.orange,
            self.green,
            self.blue,
            self.pink,
            self.brown,
            self.purple,
            self.yellow,
            self.gray,
        ][i % 9]
TFColor = _TFColor()

def session_options(enable_gpu_ram_resizing=True, enable_xla=False):
    """
    Allowing the notebook to make use of GPUs if they're available.
    
    XLA (Accelerated Linear Algebra) is a domain-specific compiler for linear 
    algebra that optimizes TensorFlow computations.
    """
    config = tf.ConfigProto()
    config.log_device_placement = True
    if enable_gpu_ram_resizing:
        # `allow_growth=True` makes it possible to connect multiple colabs to your
        # GPU. Otherwise the colab malloc's all GPU ram.
        config.gpu_options.allow_growth = True
    if enable_xla:
        # Enable on XLA. https://www.tensorflow.org/performance/xla/.
        config.graph_options.optimizer_options.global_jit_level = (
            tf.OptimizerOptions.ON_1)
    return config


def reset_sess(config=None):
    """
    Convenience function to create the TF graph & session or reset them.
    """
    if config is None:
        config = session_options(enable_gpu_ram_resizing=True, enable_xla=False)
    global sess
    tf.reset_default_graph()
    try:
        sess.close()
    except:
        pass
    sess = tf.InteractiveSession(config=config)

reset_sess()



In [23]:
# radon, county_name = preprocess_radon_dataset(*download_radon_dataset())
radon = pd.read_csv('test_data/tmp/radon/radon.csv')
radon['radon'] = (np.round(np.exp(radon.radon),1)*10).astype(int)
radon.head()


,radon,floor,county,log_uranium_ppm
0,35849128461,1,0,-0.689048
1,35849128461,0,0,-0.689048
2,39313342971440,0,0,-0.689048
3,220265,0,0,-0.689048
4,290488496652474,0,1,-0.847313


In [24]:

tfd = tfp.distributions
dtype = np.float32

log_county_uranium_ppm = radon[
    ['county', 'log_uranium_ppm']].drop_duplicates().values[:, 1]
# log_county_uranium_ppm = log_county_uranium_ppm.astype(dtype)

num_counties = len(log_county_uranium_ppm)
# county = np.int32(radon.county.values)
# floor = dtype(radon.floor.values)
# radon = dtype(radon.radon.values)

# Set the chain's start state.
initial_chain_state = [
    tf.ones([num_counties], dtype=tf.float32, name="init_sigma"),
    tf.ones([num_counties], dtype=tf.float32, name="init_gamma"),
    tf.ones([2], dtype=tf.float32, name="init_beta")
]

unconstraining_bijectors = [
    tfp.bijectors.Exp(),       # Maps a positive real to R.
    tfp.bijectors.Identity(),       # Maps a positive real to R.
    tfp.bijectors.Identity(),   # Maps [0,1] to R.  
]

def joint_log_prob2(
    my_sigma, my_gamma, my_beta, 
    floor = dtype(radon.floor.values), radon = dtype(radon.radon.values), 
    county = np.int32(radon.county.values), log_county_uranium_ppm = log_county_uranium_ppm.astype(dtype)
):
    num_counties = len(log_county_uranium_ppm)
    
#     fixed_effect_weights = tf.get_variable(
#         name='fixed_effect_weights', initializer=np.array([0., 1.], dtype=dtype))
    
#     raw_hyper_prior_scale = tf.get_variable(
#         name='raw_hyper_prior_scale',
#         initializer=np.ones(shape=num_counties, dtype=dtype))
    
    rv_sigma = tfd.Independent(
        tfd.InverseGamma(
            concentration = tf.ones(num_counties, dtype=tf.float32),
            rate = tf.ones(num_counties, dtype=tf.float32)
    ),reinterpreted_batch_ndims=1)
    
    rv_gamma = tfp.distributions.Independent(
        tfp.distributions.Normal(
            loc=tf.zeros(num_counties, dtype=tf.float32),
            scale=my_sigma),
        reinterpreted_batch_ndims=1)
    
    rv_beta = tfp.distributions.Independent(
        tfp.distributions.Normal(
            loc=tf.zeros([2], dtype=tf.float32),
            scale=tf.constant([10,10], dtype=tf.float32)),
        reinterpreted_batch_ndims=1)
    
    fixed_effects = my_beta[0] + my_beta[1] * floor
    
    random_effects = tf.gather(
        my_gamma * log_county_uranium_ppm,
        indices=tf.to_int32(county),
        axis=-1)
    linear_predictor = fixed_effects + random_effects
    
    
    lambda_ = tf.math.exp(linear_predictor)
    rv_radon = tfd.Poisson(rate=lambda_)
    
    return (
        rv_beta.log_prob(my_beta)+\
        rv_sigma.log_prob(my_sigma)+\
        rv_gamma.log_prob(my_gamma)+\
        tf.reduce_sum(rv_radon.log_prob(radon), axis=-1)
    )

def unnormalized_log_posterior(my_sigma, my_gamma, my_beta):
    return joint_log_prob2(
        my_sigma, my_gamma, my_beta, floor, radon, county, log_county_uranium_ppm
    )

with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    step_size = tf.get_variable(
        name='step_size',
        initializer=tf.constant([0.0001], dtype=tf.float32),#np.array(0.2, dtype=dtype),
        trainable=False,
        use_resource=False
    )

    
    
[
    sigma_samples,
    gamma_samples,
    beta_sample
], kernel_results = tfp.mcmc.sample_chain(
    num_results=100,
    num_burnin_steps=20,
    current_state=initial_chain_state,
    kernel=tfp.mcmc.TransformedTransitionKernel(
        inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn=joint_log_prob2,
            num_leapfrog_steps=2,
            step_size=step_size,
            step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(),
            state_gradients_are_stopped=True),
        bijector=unconstraining_bijectors))

# Initialize any created variables.
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()


In [25]:
evaluate(init_g)
evaluate(init_l)
[
    sigma_samples_,
    gamma_samples_,
    beta_sample_,
    kernel_results_
] = evaluate([
    sigma_samples,
    gamma_samples,
    beta_sample,
    kernel_results
])

    
print("acceptance rate: {}".format(
    kernel_results_.inner_results.is_accepted.mean()))
print("final step size: {}".format(
    kernel_results_.inner_results.extra.step_size_assign[-2000:].mean()))

acceptance rate: 0.0
final step size: 3.0299492209451273e-05


In [26]:
# for i in range(28):
#     plt.plot(gamma_samples_[:,i])
#     plt.show()

In [27]:
# for i in range(sigma_samples_.shape[1]):
#     plt.figure(figsize=(12.5, 15))
#     #histogram of the samples:

#     ax = plt.subplot(311)
# #     ax.set_autoscaley_on(False)

#     plt.hist(
#         sigma_samples_[:,i], histtype='stepfilled', bins=50, alpha=0.85,
#         label=r"posterior of $\lambda_1$", color=TFColor[0], density=True)
#     plt.legend(loc="upper left")
#     plt.title(r"""Posterior distributions of the variables $\lambda_1,\;\lambda_2,\;\tau$""")
# #     plt.xlim([15, 30])
#     plt.xlabel(r"$\lambda_1$ value")
#     plt.show()

In [28]:
# n_sample = len(sigma_samples_[-2000:,0])
# y_hat=[0]*n_sample

In [29]:
# radon_ = radon.iloc[0,0]
# floor_ = radon.iloc[0,1]
# county_ = int(radon.iloc[0,2])
# log_uranium_ppm_ = radon.iloc[0,3]

# for i in range(n_sample):
#     lambda_i = beta_sample_[-2000+i, ]
#     lambda_i = lambda_i[0]+lambda_i[1]*floor_
#     lambda_i = lambda_i+ log_uranium_ppm_*np.random.normal(0, gamma_samples_[-2000+i,county_])
    
#     y_hat[i] = np.random.poisson(lam=lambda_i, size = 1)
# plt.hist(y_hat)

In [30]:
# gamma_samples_[:,int(county[0])]